In [1]:
import pandas as pd

In [62]:
experiment_name = "simple_measure"

list_dataset = [
                "Lastfm",
                "Amazon-lb", 
                "QK-video",
                "Jester",
                "ML-10M",
                "ML-20M"
                ]

res_dict = {}
res_dict_rel = {}
res_dict_rec = {}

def identify_higher_corr(list_corr):
    avg, dpfr = list_corr

    if avg > dpfr:
        return "AVG"
    
    elif avg < dpfr:
        return "DPFR"
    
    elif avg == dpfr:
        return "TIE"

In [63]:
for dataset in list_dataset:
    print(dataset)
    the_corr = pd.read_csv(f"{experiment_name}/corr_simple_measure_{dataset}.csv", index_col=0)

    the_corr_sliced = the_corr.loc[:,["num_rel_items","num_unique_rec_items",]]
    the_corr_sliced.index = the_corr_sliced.index.str.replace("_our|_ori|_true","", regex=True)

    

    chosen_joint = ["IBO", "II-F", "AI-F"]

    the_corr_avg = the_corr_sliced.loc[the_corr_sliced.index.str.contains("avg")].loc[[
                                                            "P-Jain-avg", "P-Ent-avg", "P-Gini-avg",
                                                            "MAP-Jain-avg", "MAP-Ent-avg", "MAP-Gini-avg",
                                                            "R-Jain-avg", "R-Ent-avg", "R-Gini-avg",
                                                            "NDCG-Jain-avg", "NDCG-Ent-avg", "NDCG-Gini-avg"
                                                            ]]
    the_corr_avg.columns = the_corr_avg.columns + "_AVG"


    the_corr_sliced_dpfr_half = the_corr_sliced.loc[the_corr_sliced.index.str.contains("0.5$")]
    the_corr_sliced_dpfr_half.index = the_corr_sliced_dpfr_half.index.str.replace("0.5","avg")
    the_corr_sliced_dpfr_half.columns = the_corr_sliced_dpfr_half.columns + "_DPFR"

    the_corr_combined = pd.concat([the_corr_avg, the_corr_sliced_dpfr_half], axis=1)
    the_corr_combined = the_corr_combined[["num_rel_items_AVG","num_rel_items_DPFR","num_unique_rec_items_AVG","num_unique_rec_items_DPFR"]]
    the_corr_combined = the_corr_combined.reset_index().melt(id_vars="index")
    the_corr_combined["variant"] = the_corr_combined["variable"].apply(lambda x: "AVG" if "AVG" in x else "DPFR")

    the_corr_combined["index"] = the_corr_combined["index"].str.replace("-avg","")
    the_corr_combined["variable"] = the_corr_combined["variable"].str.replace("_AVG|_DPFR","", regex=True)
    the_corr_combined["value"] = the_corr_combined["value"].abs()
    corr_grouped = the_corr_combined.groupby(["index","variable"])["value"].agg(lambda x: [x for x in x])

    res_this_data = corr_grouped.apply(identify_higher_corr).value_counts().to_dict()

    corr_grouped = corr_grouped.to_frame().reset_index()

    num_rel = corr_grouped.query("variable=='num_rel_items'")
    num_rec = corr_grouped.query("variable=='num_unique_rec_items'")


    res_this_data_num_rel = num_rel.value.apply(identify_higher_corr).value_counts().to_dict()
    res_this_data_num_rec = num_rec.value.apply(identify_higher_corr).value_counts().to_dict()

    res_dict[dataset] = res_this_data
    res_dict_rel[dataset] = res_this_data_num_rel
    res_dict_rec[dataset] = res_this_data_num_rec

Lastfm
Amazon-lb
QK-video
Jester
ML-10M
ML-20M


In [64]:
result_all = (pd.DataFrame(res_dict).T.fillna(0) / 24)* 100
result_all

,AVG,DPFR,TIE
Lastfm,58.333333,37.500000,4.166667
Amazon-lb,25.000000,16.666667,58.333333
QK-video,29.166667,12.500000,58.333333
Jester,20.833333,62.500000,16.666667
ML-10M,33.333333,37.500000,29.166667
ML-20M,58.333333,41.666667,0.000000


In [69]:
result_rel = (pd.DataFrame(res_dict_rel).T.fillna(0) / 12) * 100
result_rel = result_rel[["AVG","DPFR","TIE"]]
result_rel

,AVG,DPFR,TIE
Lastfm,41.666667,50.000000,8.333333
Amazon-lb,41.666667,0.000000,58.333333
QK-video,25.000000,16.666667,58.333333
Jester,8.333333,91.666667,0.000000
ML-10M,25.000000,41.666667,33.333333
ML-20M,33.333333,66.666667,0.000000


In [70]:
result_rec = (pd.DataFrame(res_dict_rec).T.fillna(0) / 12) * 100
result_rec

,AVG,DPFR,TIE
Lastfm,75.000000,25.000000,0.000000
Amazon-lb,8.333333,33.333333,58.333333
QK-video,33.333333,8.333333,58.333333
Jester,33.333333,33.333333,33.333333
ML-10M,41.666667,33.333333,25.000000
ML-20M,83.333333,16.666667,0.000000


In [72]:
df_final = pd.concat([result_rel, result_rec, result_all], keys=["#relevant items","#unique recommended items","Overall"], axis=1)
df_final

#relevant items                       #unique recommended items  \
                      AVG       DPFR        TIE                       AVG   
Lastfm          41.666667  50.000000   8.333333                 75.000000   
Amazon-lb       41.666667   0.000000  58.333333                  8.333333   
QK-video        25.000000  16.666667  58.333333                 33.333333   
Jester           8.333333  91.666667   0.000000                 33.333333   
ML-10M          25.000000  41.666667  33.333333                 41.666667   
ML-20M          33.333333  66.666667   0.000000                 83.333333   

                                   Overall                        
                DPFR        TIE        AVG       DPFR        TIE  
Lastfm     25.000000   0.000000  58.333333  37.500000   4.166667  
Amazon-lb  33.333333  58.333333  25.000000  16.666667  58.333333  
QK-video    8.333333  58.333333  29.166667  12.500000  58.333333  
Jester     33.333333  33.333333  20.833333  62.500000  16.666667  
ML-10M     33.333333  25.000000  33.333333  37.500000  29.166667  
ML-20M     16.666667   0.000000  58.333333  41.666667   0.000000

In [80]:
print(df_final\
            .round(2)\
            .map(lambda x: '{0:.2f}'.format(x))\
            .to_latex())

\begin{tabular}{llllllllll}
\toprule
 & \multicolumn{3}{r}{#relevant items} & \multicolumn{3}{r}{#unique recommended items} & \multicolumn{3}{r}{Overall} \\
 & AVG & DPFR & TIE & AVG & DPFR & TIE & AVG & DPFR & TIE \\
\midrule
Lastfm & 41.67 & 50.00 & 8.33 & 75.00 & 25.00 & 0.00 & 58.33 & 37.50 & 4.17 \\
Amazon-lb & 41.67 & 0.00 & 58.33 & 8.33 & 33.33 & 58.33 & 25.00 & 16.67 & 58.33 \\
QK-video & 25.00 & 16.67 & 58.33 & 33.33 & 8.33 & 58.33 & 29.17 & 12.50 & 58.33 \\
Jester & 8.33 & 91.67 & 0.00 & 33.33 & 33.33 & 33.33 & 20.83 & 62.50 & 16.67 \\
ML-10M & 25.00 & 41.67 & 33.33 & 41.67 & 33.33 & 25.00 & 33.33 & 37.50 & 29.17 \\
ML-20M & 33.33 & 66.67 & 0.00 & 83.33 & 16.67 & 0.00 & 58.33 & 41.67 & 0.00 \\
\bottomrule
\end{tabular}



# Prototype

In [17]:
the_corr_combined["index"] = the_corr_combined["index"].str.replace("-avg","")
the_corr_combined["variable"] = the_corr_combined["variable"].str.replace("_AVG|_DPFR","", regex=True)
the_corr_combined["value"] = the_corr_combined["value"].abs()

In [21]:
corr_grouped = the_corr_combined.groupby(["index","variable"])["value"].agg(lambda x: [x for x in x])
corr_grouped

index      variable            
MAP-Ent    num_rel_items            [0.27, 0.5]
           num_unique_rec_items    [0.38, 0.28]
MAP-Gini   num_rel_items            [0.3, 0.25]
           num_unique_rec_items    [0.52, 0.47]
MAP-Jain   num_rel_items           [0.18, 0.08]
           num_unique_rec_items    [0.27, 0.17]
NDCG-Ent   num_rel_items            [0.07, 0.6]
           num_unique_rec_items    [0.18, 0.38]
NDCG-Gini  num_rel_items            [0.02, 0.0]
           num_unique_rec_items    [0.23, 0.22]
NDCG-Jain  num_rel_items            [0.1, 0.18]
           num_unique_rec_items    [0.05, 0.03]
P-Ent      num_rel_items           [0.28, 0.05]
           num_unique_rec_items    [0.43, 0.13]
P-Gini     num_rel_items            [0.33, 0.4]
           num_unique_rec_items    [0.55, 0.58]
P-Jain     num_rel_items           [0.18, 0.25]
           num_unique_rec_items     [0.23, 0.3]
R-Ent      num_rel_items            [0.18, 0.4]
           num_unique_rec_items    [0.33, 0.18]
R-Gini  

In [22]:
def identify_higher_corr(list_corr):
    avg, dpfr = list_corr

    if avg > dpfr:
        return "AVG"
    
    elif avg < dpfr:
        return "DPFR"
    
    elif avg == dpfr:
        return "TIE"

In [26]:
corr_grouped.apply(identify_higher_corr).value_counts().to_dict()

{'AVG': 14, 'DPFR': 9, 'TIE': 1}